### S. Zhao, Y. Gao, X. Jiang, H. Yao, T.-S. Chua, and X. Sun, “Exploring principles-of-art features for image emotion recognition,” in Proceedings of the 22nd ACM international conference on Multimedia, 2014, pp. 4756.

In [ ]:
import tensorflow as tf

import os
import re
import seaborn as sns
import numpy as np
import pandas as pd
import math
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy.random import rand
from numpy.random import randint
from matplotlib import pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

#import efficientnet.tfkeras as efn

from kaggle_datasets import KaggleDatasets
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from numpy.random import randn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = '/content/drive/My Drive/PROJECT/Image Data/train'
test = '/content/drive/My Drive/PROJECT/Image Data/test'

In [ ]:
train_datagen = ImageDataGenerator(
    width_shift_range = 0.1,        # Randomly shift the width of images by up to 10%
    height_shift_range = 0.1,       # Randomly shift the height of images by up to 10%
    horizontal_flip = True,         # Flip images horizontally at random
    rescale = 1./255,               # Rescale pixel values to be between 0 and 1
    validation_split = 0.2          # Set aside 20% of the data for validation
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255,               # Rescale pixel values to be between 0 and 1
    validation_split = 0.2          # Set aside 20% of the data for validation
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory = train,           # Directory containing the training data
    target_size = (48, 48),          # Resizes all images to 48x48 pixels
    batch_size = 64,                 # Number of images per batch
    color_mode = "grayscale",        # Converts the images to grayscale
    class_mode = "categorical",      # Classifies the images into 7 categories
    subset = "training"              # Uses the training subset of the data
)

validation_generator = validation_datagen.flow_from_directory(
    directory = test,            # Directory containing the validation data
    target_size = (48, 48),          # Resizes all images to 48x48 pixels
    batch_size = 64,                 # Number of images per batch
    color_mode = "grayscale",        # Converts the images to grayscale
    class_mode = "categorical",      # Classifies the images into 7 categories
    subset = "validation"            # Uses the validation subset of the data
)

Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [ ]:
def define_vgg16_discriminator(in_shape=(img_size,img_size,3)):
    # Relu modified to LeakyRelu
    # as described in paper works better for GAN discriminator
    # using VGG16 as backbone for this
    with strategy.scope():
        model = tf.keras.Sequential()
        tflayer = tf.keras.layers

        model.add(tflayer.Conv2D(input_shape=in_shape,filters=64,kernel_size=(3,3),padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(tflayer.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(tflayer.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.MaxPool2D(pool_size=(2,2),strides=(2,2)))

        model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.MaxPool2D(pool_size=(2,2),strides=(2,2)))

        #This is extra layer-----
        #model.add(tflayer.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation=tflayer.LeakyReLU(0.2)))
        #model.add(tflayer.MaxPool2D(pool_size=(2,2),strides=(2,2)))
        # ------------------------

        model.add(tflayer.Flatten())

        model.add(tflayer.Dense(4096, activation=tflayer.LeakyReLU(0.2)))
        model.add(tflayer.Dense(1, activation='sigmoid'))
        # compile model
        opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

        return model
    #model.add(tflayer.Dense(units=4096,activation="relu"))

In [1]:
model = define_vgg16_discriminator((img_size,img_size,3))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 192, 192, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 192, 192, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 96, 96, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 256)       

In [2]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-14 06:28:41.632917: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
Epoch 1/20
61/61 [==============================] - 186s 3s/step - loss: 2.3215 - accuracy: 0.4567 - val_loss: 1.3474 - val_accuracy: 0.4722
Epoch 2/20
61/61 [==============================] - 181s 3s/step - loss: 0.9892 - accuracy: 0.6033 - val_loss: 0.9240 - val_accuracy: 0.6173
Epoch 3/20
61/61 [==============================] - 180s 3s/step - loss: 0.9204 - accuracy: 0.6371 - val_loss: 1.2322 - val_accuracy: 0.6759
Epoch 4/20
61/61 [==============================] - 179s 3s/step - loss: 0.9090 - accuracy: 0.6433 - val_loss: 0.7130 - val_accuracy: 0.7130
Epoch 5/2